### Anotaciones Label Studio

In [1]:
ANNO_PATH = './annotations/'

In [2]:
import os
import json
import pandas as pd

In [3]:
anno_json = json.load(open(os.path.join(ANNO_PATH, 'project-1-at-2022-08-23-16-59-07845cb9.json')))

In [94]:
# https://towardsdatascience.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd
pd.json_normalize(data = anno_json, record_path = ['annotations',['result']])['original_height'].unique()[0]

8078

In [206]:
pd_table = pd.json_normalize(data = anno_json, record_path = ['annotations',['result']])

In [208]:
pd_table.shape[0]

31

In [6]:
all_x = pd_table['value.x'].to_numpy()
all_y = pd_table['value.y'].to_numpy()
all_labels = [label[0] for label in pd_table['value.keypointlabels']]

In [9]:
all_labels

['C2OT',
 'C1AE',
 'C1PE',
 'C2CE',
 'C2AI',
 'C2PI',
 'C7AS',
 'C7PS',
 'C7CE',
 'C7AI',
 'C7PI',
 'T1AS',
 'T1PS',
 'T1CE',
 'T1AI',
 'T1PI',
 'T5AS',
 'T5PS',
 'T12A',
 'T12P',
 'L1AS',
 'L1PS',
 'L4AS',
 'L4PS',
 'L4AI',
 'L4PI',
 'S1AS',
 'S1MI',
 'S1PS',
 'F1HC',
 'F2HC']

In [116]:
# obtain filename
(pd.json_normalize(data = anno_json)['file_upload'][0]).split('-')[-1]

'5195996_Anonimo013_17407677_3.jpg'

In [155]:
int(((pd.json_normalize(data = anno_json)['file_upload'][0]).split('-')[-1]).split('_')[2])

17407677

In [38]:
from os import walk
# get all json files
def get_all_json_files(path):
    filenames = next(walk(path), (None, None, []))[2] # [] if no file
    return filenames

In [115]:
get_all_json_files('./annotations/')

['project-1-at-2022-08-23-13-14-3e0784d1.json',
 'project-1-at-2022-08-23-14-32-602ee745.json',
 'project-1-at-2022-08-23-16-59-07845cb9.json']

In [127]:
from src.data.coco_class import *

In [257]:
import numpy as np

def get_keypoints(df_values, width, height):
    x = ((df_values['value.x'].to_numpy()/100.)*width).astype(int)
    y = ((df_values['value.y'].to_numpy()/100.)*height).astype(int)
    
    # reshape to [length,] to [length, 1]
    x = x.reshape((x.shape[0],1))
    y = y.reshape((y.shape[0],1))
    visibility = np.ones((x.shape[0],1))*2
    
    # return numpy array (x.shape[0],3) -> (x, y, visibility)
    return np.concatenate((x,y,visibility), axis = 1, dtype = 'float64')

In [268]:
def get_annotations(annotation_path, file_path):
    anno_json = json.load(open(os.path.join(annotation_path, file_path)))
    
    # Extract Name from annotation:
    fileName = (pd.json_normalize(data = anno_json)['file_upload'][0]).split('-')[-1]
    
    # Extract ID (Accession Number) from annotation:
    fileID = fileName.split('_')[2]
    
    # Dataframe Values
    df_values = pd.json_normalize(data = anno_json, record_path = ['annotations',['result']])
    
    # Obtain size of the image
    width = df_values['original_width'].unique()[0]
    height = df_values['original_height'].unique()[0]
    
    # Get all keypoint and transform them to [x, y, visibility = 2]
    keypoints = get_keypoints(df_values, width, height)

    return int(fileID), fileName, int(width), int(height), keypoints    

In [269]:
from src.data.coco_class import ImageCOCO
from datetime import datetime

def create_COCO(root_anno_path, annotation_files):
    currentDateTime = datetime.now().strftime('%Y-%m-%d')
    coco = COCO()
    coco.setInfo(Info(2022, "1.0.0", "Spinogram_Dataset", "", "", currentDateTime))
    coco.addLicense(License(1, 'Hospital Italiano', 'https://www.hospitalitaliano.org.ar'))
    coco.addCategorie(Categories(1, 'spine', 'spine', 
                             ['C2OT', 'C1AE', 'C1PE', 'C2CE', 'C2AI', 'C2PI', 'C7AS', 'C7PS', 'C7CE', 'C7AI', 'C7PI', 'T1AS',
                             'T1PS', 'T1CE', 'T1AI', 'T1PI', 'T5AS', 'T5PS', 'T12A', 'T12P', 'L1AS', 'L1PS', 'L4AS', 'L4PS',
                             'L4AI', 'L4PI', 'S1AS', 'S1MI', 'S1PS', 'F1HC', 'F2HC'], 
                             [[1, 2], [2, 3], [3, 4]]))
    
    annotationNumber = 1
    
    for file_path in annotation_files:
        
        fileId, fileName, width, height, keypoints = get_annotations(root_anno_path, file_path)
                    
        cocoImage = ImageCOCO(fileId, width, height, fileName, coco.getLicense().getId(), '', '', currentDateTime)
        coco.addImage(cocoImage)

        # Create annotation for image
        annotation = Annotation(annotationNumber, cocoImage.getId(), 1, [], 0.00, [], 0,
                                [coordinate for row in keypoints for coordinate in row], keypoints.shape[0])
        coco.addAnnotation(annotation)
        annotationNumber += 1
    
    return coco

In [270]:
coco_file = create_COCO('./annotations/', get_all_json_files('./annotations/'))

In [271]:
coco_file.exportJson('./annotations/coco/', 'test.json')

In [272]:
get_all_json_files('./annotations/')

['project-1-at-2022-08-23-13-14-3e0784d1.json',
 'project-1-at-2022-08-23-14-32-602ee745.json',
 'project-1-at-2022-08-23-16-59-07845cb9.json']

### Uso de las funciones desarrolladas

In [1]:
from helper.json_helper import get_all_json_files
from helper.labelStudio_helper import create_LabelStudioDataset

In [2]:
ANNO_PATH = './annotations/LabelStudio/'
IMAGES_PATH = './data/imagesLabelStudio/' 

json_annotations = get_all_json_files(ANNO_PATH)

In [3]:
LabelStudioDataset = create_LabelStudioDataset(ANNO_PATH, json_annotations)

In [4]:
LabelStudioDataset.exportJson('./data/datasetJson/', fileName='LabelStudioDataset.json')

### Prueba con Voxel 51

In [5]:
import fiftyone as fo

In [7]:
# The path to the COCO labels JSON file
labels_path = "./data/datasetJson/LabelStudioDataset.json"

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path= IMAGES_PATH,
    labels_path=labels_path,
    include_id=True,
    label_types=["keypoints"],
    extra_attrs = True,
    classes = ['spine'],
)

 100% |███████████████████| 10/10 [103.5ms elapsed, 0s remaining, 96.6 samples/s]     


In [8]:
print(dataset)

Name:        2022.09.13.11.03.33
Media type:  image
Num samples: 10
Persistent:  False
Tags:        []
Sample fields:
    id:        fiftyone.core.fields.ObjectIdField
    filepath:  fiftyone.core.fields.StringField
    tags:      fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    keypoints: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Keypoints)
    coco_id:   fiftyone.core.fields.IntField


In [9]:
dataset

Name:        2022.09.13.11.03.33
Media type:  image
Num samples: 10
Persistent:  False
Tags:        []
Sample fields:
    id:        fiftyone.core.fields.ObjectIdField
    filepath:  fiftyone.core.fields.StringField
    tags:      fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    keypoints: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Keypoints)
    coco_id:   fiftyone.core.fields.IntField

In [11]:
session = fo.launch_app(dataset)

## Busqueda de los tags
Maquina 1 - Philips: 
* (0008,0070)	Manufacturer	Philips Medical Systems
* (0008,1090)	Manufacturer Model Name	DigitalDiagnost
* (0008,0080)	Institution Name	Sociedad Italiana de Beneficencia CUARTO 6 - NEA

Maquina 2:
* (0008,0070)	Manufacturer	MED.e.COM SARL FRANCE
* (0008,1090)	Manufacturer Model Name	Duet FL
* (0008,0080)	Institution Name	HOSPITAL ITALIANO SAN JUSTO

Maquina 3:
* (0008,0070)	Manufacturer	Carestream Health
* (0008,1090)	Manufacturer Model Name	DRX-Evolution
* (0008,0080)	Institution Name	HOSPITAL ITALIANO CENTRAL

Posible tag de orientacion - (0020,0020)
* Patient direction of the rows and columns of the image. Required if image does not require Image Orientatio(Patient) (0020,0037) and Image Position (Patient) (0020,0032) or if image does not require Image Orientation (Slide) (0048,0102). May be present otherwise. See Section C.7.6.1.1.1 for further explanation.

* (0018,5101)	View Position	AP! Deja deteminar la posicion del paciente en la imagen - https://dicom.nema.org/medical/dicom/2018b/output/chtml/part03/sect_C.8.html